<a href="https://colab.research.google.com/github/Gokul0Krishna/exp/blob/main/testsite3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# bgcode

In [1]:
!unzip "/content/people- car.v791i.yolov5pytorch.zip"

Archive:  /content/people- car.v791i.yolov5pytorch.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/standing_8_11_23-1-_jpg.rf.c0c39f0ae7120ccdcc8e0ca885f2c88f.jpg  
 extracting: test/images/standing_8_11_23-10-_jpg.rf.2c58dabece38cc759e9368f02e29d694.jpg  
 extracting: test/images/standing_8_11_23-13-_jpg.rf.064185645683be0abef1520ff366b07d.jpg  
 extracting: test/images/standing_8_11_23-14-_jpg.rf.4cdbaae4f998529962869017510a8fb9.jpg  
 extracting: test/images/standing_8_11_23-18-_jpg.rf.3377c4d98ede6ced01bfe3480190bd98.jpg  
 extracting: test/images/standing_8_11_23-23-_jpg.rf.54013303c9ccda795b2779513a01f232.jpg  
 extracting: test/images/standing_8_11_23-26-_jpg.rf.cab685c3362547fdac09b62d3dc4644f.jpg  
 extracting: test/images/standing_8_11_23-27-_jpg.rf.319140471732d9f212711486eab77a0b.jpg  
 extracting: test/images/standing_8_11_23-29-_jpg.r

In [2]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17516, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 17516 (delta 8), reused 0 (delta 0), pack-reused 17494 (from 3)
Receiving objects: 100% (17516/17516), 16.60 MiB | 22.92 MiB/s, done.
Resolving deltas: 100% (11997/11997), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 

In [3]:
import cv2
import numpy as np
from pathlib import Path
from yolov5.utils.general import non_max_suppression, xywh2xyxy
import matplotlib.pyplot as plt
import torch

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
import torch
from models.experimental import attempt_load

# Load trained model
model = attempt_load('/content/carsvhuman.pt')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients, 15.8 GFLOPs


In [5]:
import torch
import numpy as np
from collections import defaultdict

def iou(box1, box2):
    """Compute IoU between two boxes: [x1, y1, x2, y2]"""
    x1 = torch.max(box1[0], box2[0])
    y1 = torch.max(box1[1], box2[1])
    x2 = torch.min(box1[2], box2[2])
    y2 = torch.min(box1[3], box2[3])

    inter_area = torch.clamp(x2 - x1, min=0) * torch.clamp(y2 - y1, min=0)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    return inter_area / (box1_area + box2_area - inter_area + 1e-6)

def compute_ap(recall, precision):
    """Compute interpolated Average Precision (AP) using trapezoidal rule"""
    mrec = torch.cat([torch.tensor([0.]), recall, torch.tensor([1.])])
    mpre = torch.cat([torch.tensor([0.]), precision, torch.tensor([0.])])

    for i in range(mpre.numel() - 1, 0, -1):
        mpre[i - 1] = torch.max(mpre[i - 1], mpre[i])

    i = (mrec[1:] != mrec[:-1]).nonzero(as_tuple=False).squeeze()
    return torch.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])


def evaluate_detections(preds, targets, iou_thresh=0.5):
    preds = torch.tensor(preds)
    targets = torch.tensor(targets)

    # Group ground truths and predictions per class
    gt_by_class = defaultdict(list)
    pred_by_class = defaultdict(list)

    for t in targets:
        if isinstance(t, (torch.Tensor, list)) and len(t) == 1:
            t = t[0]
        if isinstance(t, torch.Tensor):
            t = t.view(-1).tolist()

        img_id, cls, x1, y1, x2, y2 = t
        img_id = int(img_id)
        cls = int(cls)
        box = torch.tensor([x1, y1, x2, y2]).float()
        gt_by_class[cls].append((img_id, box))

    for p in preds:
        if isinstance(p, (torch.Tensor, list)) and len(p) == 1:
            p = p[0]
        if isinstance(p, torch.Tensor):
            p = p.view(-1).tolist()

        img_id, cls, conf, x1, y1, x2, y2 = p
        img_id = int(img_id)
        cls = int(cls)
        conf = float(conf)
        box = torch.tensor([x1, y1, x2, y2]).float()
        pred_by_class[cls].append((img_id, conf, box))



    results = {}
    aps = []

    for cls in sorted(gt_by_class.keys()):
        gt = gt_by_class[cls]
        pred = sorted(pred_by_class.get(cls, []), key=lambda x: -x[1])  # sort by conf

        gt_used = defaultdict(set)  # img_id -> set of matched GT indices
        tp = torch.zeros(len(pred))
        fp = torch.zeros(len(pred))

        for i, (img_id, conf, pred_box) in enumerate(pred):
            gts = [b for j, b in enumerate(gt) if b[0] == img_id]
            ious = [iou(pred_box, b[1]) for b in gts] if gts else []

            if ious and max(ious) >= iou_thresh:
                max_iou_idx = int(torch.argmax(torch.tensor(ious)))
                if max_iou_idx not in gt_used[img_id]:
                    tp[i] = 1
                    gt_used[img_id].add(max_iou_idx)
                else:
                    fp[i] = 1
            else:
                fp[i] = 1

        tp_cum = torch.cumsum(tp, dim=0)
        fp_cum = torch.cumsum(fp, dim=0)
        recall = tp_cum / (len(gt) + 1e-6)
        precision = tp_cum / (tp_cum + fp_cum + 1e-6)

        ap = compute_ap(recall, precision)
        aps.append(ap)

        results[cls] = {
            "precision": precision[-1].item() if len(precision) else 0.0,
            "recall": recall[-1].item() if len(recall) else 0.0,
            "f1": (2 * precision[-1] * recall[-1] / (precision[-1] + recall[-1] + 1e-6)).item() if len(precision) else 0.0,
            "ap50": ap.item()
        }

    mAP = torch.tensor(aps).mean().item() if aps else 0.0
    return results, mAP


In [6]:
def load_image(path, img_size=640):
    img0 = cv2.imread(str(path))  # BGR
    img = cv2.resize(img0, (img_size, img_size))
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3xHxW
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).float()
    img /= 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
    return img.to(device), img0

image_dir = Path('/content/test/images')
label_dir = Path('/content/test/labels')

def scale_coords(img1_shape, coords, img0_shape, ratio_pad=None):
      """Rescale coords (xyxy) from img1_shape to img0_shape"""
      if ratio_pad is None:
          # Calculate gain and pad
          gain = min(img1_shape[0] / img0_shape[0], img1_shape[1] / img0_shape[1])  # gain = old / new
          pad = (img1_shape[1] - img0_shape[1] * gain) / 2, (img1_shape[0] - img0_shape[0] * gain) / 2  # x, y
      else:
          gain = ratio_pad[0]
          pad = ratio_pad[1]

      coords[:, [0, 2]] -= pad[0]  # x padding
      coords[:, [1, 3]] -= pad[1]  # y padding
      coords[:, :4] /= gain
      coords[:, :4] = coords[:, :4].clamp(min=0)
      return coords


def pedict(img_tensor, img0, img_path, image_id):
    model.eval()
    preds_for_img = []
    targets_for_img = []

    with torch.no_grad():
        pred = model(img_tensor)[0]  # Assuming model returns (pred, ...)
        pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.5)[0]

        if pred is not None and len(pred):
            pred[:, :4] = scale_coords(img_tensor.shape[2:], pred[:, :4], img0.shape).round()
            for *box, conf, cls in pred.tolist():
                preds_for_img.append([image_id, int(cls), conf, *box])

    # Load ground truth
    label_path = label_dir / (img_path.stem + '.txt')
    h, w = img0.shape[:2]
    if label_path.exists():
        with open(label_path, 'r') as f:
            for line in f:
                cls, xc, yc, bw, bh = map(float, line.strip().split())
                x1 = (xc - bw / 2) * w
                y1 = (yc - bh / 2) * h
                x2 = (xc + bw / 2) * w
                y2 = (yc + bh / 2) * h
                targets_for_img.append([image_id, int(cls), x1, y1, x2, y2])

    return preds_for_img, targets_for_img


In [7]:
from skimage.feature import local_binary_pattern
def er(image):
  grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
  grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
  grad_mag = np.sqrt(grad_x**2 + grad_y**2)
  return np.mean(grad_mag)

def obj_found(image):
  image = (image * 255).astype(np.uint8)
  params = cv2.SimpleBlobDetector_Params()
  params.filterByArea = True
  params.minArea = 10  # Minimum blob size (adjust for small objects)
  params.maxArea = 1000
  detector = cv2.SimpleBlobDetector_create(params)
  keypoints = detector.detect(image)
  return len(keypoints)

def ed(image):
    local_var = cv2.Laplacian(image, cv2.CV_32F).var()#The Laplacian operator highlights regions of rapid intensity change (edges),High variance = many strong edges = likely not occluded
    return local_var / (np.mean(image)+ 1e-6)

def is_blurry(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

def brightness(img):
    brightness = np.mean(img)
    return brightness

def complexity(image):
  image = np.array(image).reshape(len(image), -1)
  lbp = local_binary_pattern(image, 8, 1, method="uniform")#greater the no. more complex the image
  hist, _ = np.histogram(lbp, bins=8+2, range=(0, 8+2))
  hist = hist.astype(np.float32) / hist.sum()
  grad_mag=-np.sum(hist * np.log(hist + 1e-6))
  return grad_mag


In [ ]:
# classess : ['-', 'It includes 627 images.', 'This dataset was exported via roboflow.ai on June 15- 2021 at 6-27 PM GMT', 'Vehicles are annotated in YOLO v5 PyTorch format.', 'Vehicles-OpenImages - v1 416x416', 'person - v1 2023-11-19 1-21pm']

# control

In [8]:
image_id = 0
count=0
all_preds = []
all_targets = []
for img_path in image_dir.glob('*.jpg'):
        img_tensor, img0 = load_image(img_path)  # Should return a tensor [1,3,H,W] and original image
        preds, targets = pedict(img_tensor, img0, img_path, image_id)
        all_preds.append(preds)
        all_targets.append(targets)
        count+=1
        image_id += 1

all_preds_flat = [item for sublist in all_preds for item in sublist]
all_targets_flat = [item for sublist in all_targets for item in sublist]

# Clean predictions
all_preds_clean = [p for p in all_preds_flat if len(p) == 7]
all_targets_clean = [t for t in all_targets_flat if len(t) == 6]

results, mAP50 = evaluate_detections(all_preds_clean, all_targets_clean, iou_thresh=0.6)

for cls, metrics in results.items():
    print(f"Class {cls}:")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall:    {metrics['recall']:.4f}")
    print(f"  F1 Score:  {metrics['f1']:.4f}")
    print(f"  AP@0.5:    {metrics['ap50']:.4f}")
print(f"  count:     {count}")
print(f"\nMean AP@0.5 (mAP50): {mAP50:.4f}")

Class 5:
  Precision: 0.8958
  Recall:    0.9149
  F1 Score:  0.9053
  AP@0.5:    0.9053
  count:     35

Mean AP@0.5 (mAP50): 0.9053


# edge density

In [9]:
image_id = 0
count=0
all_preds = []
all_targets = []
for img_path in image_dir.glob('*.jpg'):
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28,28))
    img = np.array(img, dtype=np.float32) / 255.0
    img = img.flatten()
    if ed(img) < 0.41 :
        img_tensor, img0 = load_image(img_path)  # Should return a tensor [1,3,H,W] and original image
        preds, targets = pedict(img_tensor, img0, img_path, image_id)
        all_preds.append(preds)
        all_targets.append(targets)
        count+=1
    image_id += 1

all_preds_flat = [item for sublist in all_preds for item in sublist]
all_targets_flat = [item for sublist in all_targets for item in sublist]

# Clean predictions
all_preds_clean = [p for p in all_preds_flat if len(p) == 7]
all_targets_clean = [t for t in all_targets_flat if len(t) == 6]

results, mAP50 = evaluate_detections(all_preds_clean, all_targets_clean, iou_thresh=0.6)

for cls, metrics in results.items():
    print(f"Class {cls}:")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall:    {metrics['recall']:.4f}")
    print(f"  F1 Score:  {metrics['f1']:.4f}")
    print(f"  AP@0.5:    {metrics['ap50']:.4f}")
print(f"  count:     {count}")
print(f"\nMean AP@0.5 (mAP50): {mAP50:.4f}")

Class 5:
  Precision: 0.8958
  Recall:    0.9149
  F1 Score:  0.9053
  AP@0.5:    0.9053
  count:     35

Mean AP@0.5 (mAP50): 0.9053


In [10]:
image_id = 0
count=0
all_preds = []
all_targets = []
for img_path in image_dir.glob('*.jpg'):
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28,28))
    img = np.array(img, dtype=np.float32) / 255.0
    img = img.flatten()
    if ed(img) >  0.41 :
        img_tensor, img0 = load_image(img_path)  # Should return a tensor [1,3,H,W] and original image
        preds, targets = pedict(img_tensor, img0, img_path, image_id)
        all_preds.append(preds)
        all_targets.append(targets)
        count+=1
    image_id += 1

all_preds_flat = [item for sublist in all_preds for item in sublist]
all_targets_flat = [item for sublist in all_targets for item in sublist]

# Clean predictions
all_preds_clean = [p for p in all_preds_flat if len(p) == 7]
all_targets_clean = [t for t in all_targets_flat if len(t) == 6]

results, mAP50 = evaluate_detections(all_preds_clean, all_targets_clean, iou_thresh=0.6)

for cls, metrics in results.items():
    print(f"Class {cls}:")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall:    {metrics['recall']:.4f}")
    print(f"  F1 Score:  {metrics['f1']:.4f}")
    print(f"  AP@0.5:    {metrics['ap50']:.4f}")
print(f"  count:     {count}")
print(f"\nMean AP@0.5 (mAP50): {mAP50:.4f}")

  count:     0

Mean AP@0.5 (mAP50): 0.0000


# complexity

In [13]:
image_id = 0
count=0
all_preds = []
all_targets = []
for img_path in image_dir.glob('*.jpg'):
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28,28))
    img = np.array(img, dtype=np.float32) / 255.0
    img = img.flatten()
    if complexity(img)<0.89 or complexity(img)>1.34:
        img_tensor, img0 = load_image(img_path)  # Should return a tensor [1,3,H,W] and original image
        preds, targets = pedict(img_tensor, img0, img_path, image_id)
        all_preds.append(preds)
        all_targets.append(targets)
        count+=1
    image_id += 1

all_preds_flat = [item for sublist in all_preds for item in sublist]
all_targets_flat = [item for sublist in all_targets for item in sublist]

# Clean predictions
all_preds_clean = [p for p in all_preds_flat if len(p) == 7]
all_targets_clean = [t for t in all_targets_flat if len(t) == 6]

results, mAP50 = evaluate_detections(all_preds_clean, all_targets_clean, iou_thresh=0.6)

for cls, metrics in results.items():
    print(f"Class {cls}:")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall:    {metrics['recall']:.4f}")
    print(f"  F1 Score:  {metrics['f1']:.4f}")
    print(f"  AP@0.5:    {metrics['ap50']:.4f}")
print(f"  count:     {count}")
print(f"\nMean AP@0.5 (mAP50): {mAP50:.4f}")

Class 5:
  Precision: 0.7000
  Recall:    0.7778
  F1 Score:  0.7368
  AP@0.5:    0.7500
  count:     7

Mean AP@0.5 (mAP50): 0.7500


In [14]:
image_id = 0
count=0
all_preds = []
all_targets = []
for img_path in image_dir.glob('*.jpg'):
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28,28))
    img = np.array(img, dtype=np.float32) / 255.0
    img = img.flatten()
    if complexity(img)>0.89 and complexity(img)<1.34:
        img_tensor, img0 = load_image(img_path)  # Should return a tensor [1,3,H,W] and original image
        preds, targets = pedict(img_tensor, img0, img_path, image_id)
        all_preds.append(preds)
        all_targets.append(targets)
        count+=1
    image_id += 1

all_preds_flat = [item for sublist in all_preds for item in sublist]
all_targets_flat = [item for sublist in all_targets for item in sublist]

# Clean predictions
all_preds_clean = [p for p in all_preds_flat if len(p) == 7]
all_targets_clean = [t for t in all_targets_flat if len(t) == 6]

results, mAP50 = evaluate_detections(all_preds_clean, all_targets_clean, iou_thresh=0.6)

for cls, metrics in results.items():
    print(f"Class {cls}:")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall:    {metrics['recall']:.4f}")
    print(f"  F1 Score:  {metrics['f1']:.4f}")
    print(f"  AP@0.5:    {metrics['ap50']:.4f}")
print(f"  count:     {count}")
print(f"\nMean AP@0.5 (mAP50): {mAP50:.4f}")

Class 5:
  Precision: 0.9474
  Recall:    0.9474
  F1 Score:  0.9474
  AP@0.5:    0.9416
  count:     28

Mean AP@0.5 (mAP50): 0.9416


# brightness

In [15]:
image_id = 0
count=0
all_preds = []
all_targets = []
for img_path in image_dir.glob('*.jpg'):
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28,28))
    img = np.array(img, dtype=np.float32) / 255.0
    img = img.flatten()
    if brightness(img) < 0.11 or brightness(img)>1.17:
        img_tensor, img0 = load_image(img_path)  # Should return a tensor [1,3,H,W] and original image
        preds, targets = pedict(img_tensor, img0, img_path, image_id)
        all_preds.append(preds)
        all_targets.append(targets)
        count+=1
    image_id += 1

all_preds_flat = [item for sublist in all_preds for item in sublist]
all_targets_flat = [item for sublist in all_targets for item in sublist]

# Clean predictions
all_preds_clean = [p for p in all_preds_flat if len(p) == 7]
all_targets_clean = [t for t in all_targets_flat if len(t) == 6]

results, mAP50 = evaluate_detections(all_preds_clean, all_targets_clean, iou_thresh=0.6)

for cls, metrics in results.items():
    print(f"Class {cls}:")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall:    {metrics['recall']:.4f}")
    print(f"  F1 Score:  {metrics['f1']:.4f}")
    print(f"  AP@0.5:    {metrics['ap50']:.4f}")
print(f"  count:     {count}")
print(f"\nMean AP@0.5 (mAP50): {mAP50:.4f}")

  count:     0

Mean AP@0.5 (mAP50): 0.0000


In [16]:
image_id = 0
count=0
all_preds = []
all_targets = []
for img_path in image_dir.glob('*.jpg'):
    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28,28))
    img = np.array(img, dtype=np.float32) / 255.0
    img = img.flatten()
    if brightness(img) > 0.11 and brightness(img)<1.17:
        img_tensor, img0 = load_image(img_path)  # Should return a tensor [1,3,H,W] and original image
        preds, targets = pedict(img_tensor, img0, img_path, image_id)
        all_preds.append(preds)
        all_targets.append(targets)
        count+=1
    image_id += 1

all_preds_flat = [item for sublist in all_preds for item in sublist]
all_targets_flat = [item for sublist in all_targets for item in sublist]

# Clean predictions
all_preds_clean = [p for p in all_preds_flat if len(p) == 7]
all_targets_clean = [t for t in all_targets_flat if len(t) == 6]

results, mAP50 = evaluate_detections(all_preds_clean, all_targets_clean, iou_thresh=0.6)

for cls, metrics in results.items():
    print(f"Class {cls}:")
    print(f"  Precision: {metrics['precision']:.4f}")
    print(f"  Recall:    {metrics['recall']:.4f}")
    print(f"  F1 Score:  {metrics['f1']:.4f}")
    print(f"  AP@0.5:    {metrics['ap50']:.4f}")
print(f"  count:     {count}")
print(f"\nMean AP@0.5 (mAP50): {mAP50:.4f}")

Class 5:
  Precision: 0.8958
  Recall:    0.9149
  F1 Score:  0.9053
  AP@0.5:    0.9053
  count:     35

Mean AP@0.5 (mAP50): 0.9053
